# Analysis of MS trial data from clinicaltrials.gov
For the raw data preprocessing, see ``01_preprocessing.ipynb``.

## Data sources
* Clinical trial data: https://www.clinicaltrials.gov/
* Socioeconomic and geographic data: https://www.naturalearthdata.com/
* HDI data: https://unstats.un.org/, http://data.un.org/Default.aspx
## Contents


### Part 1 - trial data

#### MS trial data further processing
* Filter the trial data by funder type, intervention type, and study type
* Remove trials without detailed location information
* Remove the single early phase 1 trial

#### Base level stats
* Number of trials, sites, years, sponsors, ...


### Part 2 - additional socioeconomic data

#### Socioeconomic data from Natural Earth
* Filter relevant continent
* Create high-level development and income category groups
* Check mapping to trial data

#### Human Development Index (HDI) data
* Add HDI level groups
* Add missing countries manually
* Check mapping to trial data


### Part 3 - trial and site counts

#### Create dataframes with trial and site counts
Trials and sites per:
* Continent
* Region
* Country
* HDI category
* Income category
* Economical development category

Normalized data:
* Data per capita
* Normalize over phases or groups
* Compute expected sites per capita and difference to actual number (factor, log of factor)

In [1]:
%cd ..

c:\Users\stefa\Desktop\clinical-trial-landscape


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext blackcellmagic

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tools import counters

# Part 1 - trial data

In [5]:
raw_ms_trial_data = pd.read_excel("data/intermediate/ms_trials_long.xlsx")

In [6]:
raw_ms_trial_data.head()

,nct_number,study_title,study_url,study_status,conditions,condition_category,interventions,intervention_type,sponsor,collaborators,...,completion_date,start_year,primary_completion_year,completion_year,locations,many_locations_flag,country_norm,country_ISO,country_continent,n_sites
0,NCT04447937,Immunodeficiency in MS,https://beta.clinicaltrials.gov/study/NCT04447937,UNKNOWN,Multiple Sclerosis|Hypogammaglobulinemia|Immun...,MS,OTHER: No Interventions,OTHER,Advanced Neurosciences Institute,Novel Pharmaceutics Institute,...,2021-07-30,2020.0,2021.0,2021.0,"Advanced Neurosciences Institute, Franklin, Te...",0.0,United States,USA,North America,1.0
1,NCT04447937,Immunodeficiency in MS,https://beta.clinicaltrials.gov/study/NCT04447937,UNKNOWN,Multiple Sclerosis|Hypogammaglobulinemia|Immun...,MS,OTHER: No Interventions,OTHER,Advanced Neurosciences Institute,Novel Pharmaceutics Institute,...,2021-07-30,2020.0,2021.0,2021.0,"Advanced Neurosciences Institute, Franklin, Te...",0.0,United States,USA,North America,1.0
2,NCT00942214,Biomarkers and Response to Natalizumab for Mul...,https://beta.clinicaltrials.gov/study/NCT00942214,COMPLETED,Multiple Sclerosis,MS,DRUG: Natalizumab,DRUG,"University Hospital, Toulouse",NaN,...,2011-03,2009.0,2011.0,2011.0,"service de neurologie, hôpital Purpan, Toulous...",0.0,France,FRA,Europe,1.0
3,NCT05562414,Transient and Immediate Motor Effects of Exerc...,https://beta.clinicaltrials.gov/study/NCT05562414,RECRUITING,"Multiple Sclerosis, Chronic Progressive|High-I...",PMS,BEHAVIORAL: HIIT|BEHAVIORAL: MCT,BEHAVIORAL,Klinik Valens,NaN,...,2023-04-01,2022.0,2023.0,2023.0,"Klinik Valens, Valens rehabilitation clinic, V...",0.0,Switzerland,CHE,Europe,1.0
4,NCT05562414,Transient and Immediate Motor Effects of Exerc...,https://beta.clinicaltrials.gov/study/NCT05562414,RECRUITING,"Multiple Sclerosis, Chronic Progressive|High-I...",PMS,BEHAVIORAL: HIIT|BEHAVIORAL: MCT,BEHAVIORAL,Klinik Valens,NaN,...,2023-04-01,2022.0,2023.0,2023.0,"Klinik Valens, Valens rehabilitation clinic, V...",0.0,Switzerland,CHE,Europe,1.0


## Preprocess and filter
For the present analysis, we only include industry sponsored interventional drug trials of sufficient data quality and geographical resolution.

### Industry sponsored interventional drug trials only

In [7]:
raw_ms_trial_data[["intervention_type"]].drop_duplicates()

,intervention_type
0,OTHER
2,DRUG
3,BEHAVIORAL
33,NaN
40,DEVICE
59,PROCEDURE
75,DIETARY_SUPPLEMENT
91,GENETIC
171,BIOLOGICAL
218,DIAGNOSTIC_TEST


In [8]:
raw_ms_trial_data[["funder_type"]].drop_duplicates()

,funder_type
0,OTHER
5,INDUSTRY
86,NIH
91,OTHER_GOV
797,NETWORK
1710,INDIV
1912,FED


In [9]:
raw_ms_trial_data[["study_type"]].drop_duplicates()

,study_type
0,OBSERVATIONAL
2,INTERVENTIONAL
5056,EXPANDED_ACCESS


In [10]:
ms_trial_data = raw_ms_trial_data[
    (raw_ms_trial_data["funder_type"] == "INDUSTRY")
    & (raw_ms_trial_data["intervention_type"] == "DRUG")
    & (raw_ms_trial_data["study_type"] == "INTERVENTIONAL")
].copy()

### Only trials where phases are specified and locations are provided in detail

In [11]:
ms_trial_data = ms_trial_data[
    (~ms_trial_data["phase"].isna()) & (~ms_trial_data["locations"].isna()) & (ms_trial_data["many_locations_flag"] == 0)
].copy()

In [12]:
ms_trial_data[["nct_number", "phase"]].groupby("phase").count().reset_index()

,phase,nct_number
0,EARLY_PHASE1,1
1,PHASE1,138
2,PHASE2,850
3,PHASE3,3108
4,PHASE4,499


### Remove the single early phase 1 trial

In [13]:
ms_trial_data = ms_trial_data[ms_trial_data["phase"] != "EARLY_PHASE1"].copy()

In [14]:
ms_trial_data = ms_trial_data[(~ms_trial_data["phase"].isna())].copy()

### Drop columns that are not required for the present analysis

In [15]:
ms_trial_data = (
    ms_trial_data[
        [
            "nct_number",
            "study_status",
            "sponsor",
            "phase",
            "enrollment",
            "primary_purpose",
            "start_year",
            "primary_completion_year",
            "completion_year",
            "country_norm",
            "country_ISO",
            "country_continent",
            "n_sites",
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
    .copy()
)

In [16]:
ms_trial_data

,nct_number,study_status,sponsor,phase,enrollment,primary_purpose,start_year,primary_completion_year,completion_year,country_norm,country_ISO,country_continent,n_sites
0,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Belgium,BEL,Europe,3.0
1,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Canada,CAN,North America,3.0
2,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Czech Republic,CZE,Europe,3.0
3,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,France,FRA,Europe,5.0
4,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Germany,DEU,Europe,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3236,NCT05119569,ACTIVE_NOT_RECRUITING,Hoffmann-La Roche,PHASE2,109.0,TREATMENT,2022.0,2023.0,2026.0,United States,USA,North America,2.0
3237,NCT05417269,RECRUITING,Imcyse SA,PHASE1,150.0,TREATMENT,2022.0,2025.0,2025.0,Moldova,MDA,Europe,1.0
3238,NCT05417269,RECRUITING,Imcyse SA,PHASE2,150.0,TREATMENT,2022.0,2025.0,2025.0,Moldova,MDA,Europe,1.0
3239,NCT01487096,COMPLETED,Sanofi,PHASE2,179.0,TREATMENT,2001.0,2003.0,2003.0,Canada,CAN,North America,1.0


## Base level stats

In [17]:
n_trials_total = len(ms_trial_data["nct_number"].drop_duplicates())

In [18]:
print("Number of trials:", n_trials_total)

Number of trials: 435


In [19]:
print("Number of countries:", len(ms_trial_data["country_norm"].drop_duplicates()))

Number of countries: 79


**Note:** Hong Kong is listed as separate country in our data set for historical compatibility (e.g. if one would want to combine trial data with older socioeconomic data). However, since for the present analysis we only use the most recent data, trials in Hong Kong will be assigned to China, and the publication thus reports only 78 countries.

### Study status
We do not filter by status; this is just an overview.

In [20]:
ms_trial_data[["nct_number", "study_status"]].drop_duplicates().groupby(
    "study_status"
).count().reset_index().rename(columns={"nct_number": "n_trials"}).sort_values(
    "n_trials", ascending=False
)

,study_status,n_trials
1,COMPLETED,279
5,TERMINATED,65
3,RECRUITING,43
0,ACTIVE_NOT_RECRUITING,34
6,UNKNOWN,6
7,WITHDRAWN,5
2,NOT_YET_RECRUITING,2
4,SUSPENDED,1


### Most prominent sponsors

In [21]:
print("Number of sponsors:", len(ms_trial_data["sponsor"].drop_duplicates()))

Number of sponsors: 94


In [22]:
ms_trial_data[["nct_number", "sponsor"]].drop_duplicates().groupby(
    "sponsor"
).count().reset_index().rename(columns={"nct_number": "n_trials"}).sort_values(
    "n_trials", ascending=False
).head()

,sponsor,n_trials
19,Biogen,80
71,Novartis Pharmaceuticals,47
80,Sanofi,31
49,Hoffmann-La Roche,26
45,GlaxoSmithKline,16


### Trials per phase

How many phases within the same trial?

In [23]:
ms_trial_data[["nct_number", "phase"]].drop_duplicates().groupby(
    "nct_number"
).count().reset_index().rename(columns={"phase": "n_phases_included"}).groupby(
    "n_phases_included"
).count().reset_index().rename(
    columns={"nct_number": "n_trials"}
)

,n_phases_included,n_trials
0,1,421
1,2,14


How many trials for each phase?

In [24]:
ms_trial_data[["nct_number", "phase"]].drop_duplicates().groupby(
    "phase"
).count().reset_index().rename(columns={"nct_number": "n_trials"})

,phase,n_trials
0,PHASE1,64
1,PHASE2,123
2,PHASE3,183
3,PHASE4,79


### Trials per continent
Note: trials can have sites on multiple continents; i.e. here we count in how many trials a continent is included.

In [25]:
ms_trial_data[["nct_number", "country_continent"]].drop_duplicates().groupby(
    "nct_number"
).count().reset_index().rename(columns={"country_continent": "n_continents"}).groupby(
    "n_continents"
).count().reset_index().rename(
    columns={"nct_number": "n_trials"}
)

,n_continents,n_trials
0,1,246
1,2,74
2,3,42
3,4,34
4,5,27
5,6,12


Interpretation: 42 trials have at least one site on 3 continents.

In [26]:
ms_trial_data[["nct_number", "country_continent"]].drop_duplicates().groupby(
    "country_continent"
).count().reset_index().rename(columns={"nct_number": "n_studies"})

,country_continent,n_studies
0,Africa,29
1,Asia,140
2,Europe,297
3,North America,267
4,Oceania,76
5,South America,54


Interpretation: 29 trials have at leas one site in Africa.

### Starting year

In [27]:
trials_per_start_year = (
    ms_trial_data[["nct_number", "start_year"]]
    .drop_duplicates()
    .groupby("start_year")
    .count()
    .reset_index()
    .astype(int)
    .rename(columns={"nct_number": "n_trials"})
)

In [28]:
trials_per_start_year.head()

,start_year,n_trials
0,1994,1
1,1999,1
2,2001,7
3,2002,12
4,2003,14


#### Get counts per decade

In [29]:
trials_per_start_year[trials_per_start_year["start_year"] < 2000]["n_trials"].sum()

2

In [30]:
trials_per_start_year[
    (trials_per_start_year["start_year"] >= 2000)
    & (trials_per_start_year["start_year"] < 2010)
]["n_trials"].sum()

136

In [31]:
trials_per_start_year[
    (trials_per_start_year["start_year"] >= 2010)
    & (trials_per_start_year["start_year"] < 2020)
]["n_trials"].sum()

226

In [32]:
trials_per_start_year[trials_per_start_year["start_year"] >= 2020]["n_trials"].sum()

68

# Part 2 - additional socioeconomic data

#### Socioeconomic data from Natural Earth
* Filter relevant continent
* Create high-level development and income category groups
* Check mapping to trial data

#### Human Development Index (HDI) data
* Add HDI level groups
* Add missing countries manually
* Check mapping to trial data


## Load and prepare data from Natural Earth
https://www.naturalearthdata.com/

In [33]:
world_data = gpd.read_file("data/source/naturalearth/110m_cultural/ne_110m_admin_0_countries.shx")

In [34]:
world_data.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,1,6,Fiji,FJI,0,2,Sovereign country,1,Fiji,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Admin-0 country,1,3,United Republic of Tanzania,TZA,0,2,Sovereign country,1,United Republic of Tanzania,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Admin-0 country,1,7,Western Sahara,SAH,0,2,Indeterminate,1,Western Sahara,...,Unrecognized,Unrecognized,Unrecognized,NaN,NaN,Unrecognized,NaN,NaN,NaN,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Admin-0 country,1,2,Canada,CAN,0,2,Sovereign country,1,Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,Admin-0 country,1,2,United States of America,US1,1,2,Country,1,United States of America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [35]:
world_data = (
    world_data[
        [
            "NAME",
            "ADM0_A3",
            "CONTINENT",
            "REGION_UN",
            "SUBREGION",
            "POP_EST",
            "ECONOMY",
            "INCOME_GRP",
        ]
    ]
    .rename(
        columns={
            "NAME": "country_name",
            "ADM0_A3": "country_ISO",
            "CONTINENT": "continent",
            "REGION_UN": "region_UN",
            "SUBREGION": "subregion",
            "POP_EST": "population_estimate",
            "ECONOMY": "economy_level",
            "INCOME_GRP": "income_group",
        }
    )
    .copy()
)

In [36]:
world_data

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group
0,Fiji,FJI,Oceania,Oceania,Melanesia,889953.0,6. Developing region,4. Lower middle income
1,Tanzania,TZA,Africa,Africa,Eastern Africa,58005463.0,7. Least developed region,5. Low income
2,W. Sahara,SAH,Africa,Africa,Northern Africa,603253.0,7. Least developed region,5. Low income
3,Canada,CAN,North America,Americas,Northern America,37589262.0,1. Developed region: G7,1. High income: OECD
4,United States of America,USA,North America,Americas,Northern America,328239523.0,1. Developed region: G7,1. High income: OECD
...,...,...,...,...,...,...,...,...
172,Serbia,SRB,Europe,Europe,Southern Europe,6944975.0,6. Developing region,3. Upper middle income
173,Montenegro,MNE,Europe,Europe,Southern Europe,622137.0,6. Developing region,3. Upper middle income
174,Kosovo,KOS,Europe,Europe,Southern Europe,1794248.0,6. Developing region,4. Lower middle income
175,Trinidad and Tobago,TTO,North America,Americas,Caribbean,1394973.0,6. Developing region,2. High income: nonOECD


### Check mapping to trial data

#### Any countries without geo data?

In [37]:
[
    country_iso
    for country_iso in list(ms_trial_data["country_ISO"].drop_duplicates())
    if country_iso not in list(world_data["country_ISO"].drop_duplicates())
]

['HKG', 'SGP']

We will map Hong Kong to China, and add population etc. for Singapore manually.

##### HKG to China

In [38]:
ms_trial_data[ms_trial_data["country_norm"].str.startswith("China")].head(1)

,nct_number,study_status,sponsor,phase,enrollment,primary_purpose,start_year,primary_completion_year,completion_year,country_norm,country_ISO,country_continent,n_sites
252,NCT02201108,ACTIVE_NOT_RECRUITING,"Genzyme, a Sanofi Company",PHASE3,166.0,TREATMENT,2014.0,2019.0,2025.0,China,CHN,Asia,12.0


In [39]:
ms_trial_data["country_ISO"] = ms_trial_data.apply(lambda row: "CHN" if row["country_ISO"] == "HKG" else row["country_ISO"], axis=1)
ms_trial_data["country_norm"] = ms_trial_data.apply(lambda row: "China" if row["country_norm"] == "Hong Kong" else row["country_norm"], axis=1)

In [40]:
ms_trial_data.head()

,nct_number,study_status,sponsor,phase,enrollment,primary_purpose,start_year,primary_completion_year,completion_year,country_norm,country_ISO,country_continent,n_sites
0,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Belgium,BEL,Europe,3.0
1,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Canada,CAN,North America,3.0
2,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Czech Republic,CZE,Europe,3.0
3,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,France,FRA,Europe,5.0
4,NCT00883337,COMPLETED,Sanofi,PHASE3,324.0,TREATMENT,2009.0,2011.0,2015.0,Germany,DEU,Europe,11.0


In [41]:
ms_trial_data[ms_trial_data["country_norm"].str.startswith("Hong")]

,nct_number,study_status,sponsor,phase,enrollment,primary_purpose,start_year,primary_completion_year,completion_year,country_norm,country_ISO,country_continent,n_sites


##### Singapore
Source: Wikipedia

In [42]:
singapore_manual = pd.DataFrame(
    [
        {
            "country_name": "Singapore",
            "country_ISO": "SGP",
            "continent": "Asia",
            "region_UN": "Asia",
            "subregion": "South-Eastern Asia",
            "population_estimate": 5_637_000,
            "economy_level": "2. Developed region: nonG7",
            "income_group": "2. High income: nonOECD",
        }
    ]
)

In [43]:
singapore_manual

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group
0,Singapore,SGP,Asia,Asia,South-Eastern Asia,5637000,2. Developed region: nonG7,2. High income: nonOECD


In [44]:
world_data = pd.concat([world_data, singapore_manual]).reset_index(drop=True)

In [45]:
world_data.tail()

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group
173,Montenegro,MNE,Europe,Europe,Southern Europe,622137.0,6. Developing region,3. Upper middle income
174,Kosovo,KOS,Europe,Europe,Southern Europe,1794248.0,6. Developing region,4. Lower middle income
175,Trinidad and Tobago,TTO,North America,Americas,Caribbean,1394973.0,6. Developing region,2. High income: nonOECD
176,S. Sudan,SDS,Africa,Africa,Eastern Africa,11062113.0,7. Least developed region,5. Low income
177,Singapore,SGP,Asia,Asia,South-Eastern Asia,5637000.0,2. Developed region: nonG7,2. High income: nonOECD


#### Any mismatches?

In [46]:
trial_geo_mapped_countries = pd.merge(
    left=ms_trial_data[["country_norm", "country_ISO"]].drop_duplicates(),
    right=world_data[["country_name", "country_ISO"]].drop_duplicates(),
    on="country_ISO",
    how="left",
)

In [47]:
trial_geo_mapped_countries[trial_geo_mapped_countries["country_norm"] != trial_geo_mapped_countries["country_name"]]

,country_norm,country_ISO,country_name
2,Czech Republic,CZE,Czechia
15,United States,USA,United States of America
53,Republic of Korea,KOR,South Korea
55,Bosnia and Herzegovina,BIH,Bosnia and Herz.
77,Dominican Republic,DOM,Dominican Rep.


### Regions/continents without any trials?

In [48]:
covered_regions = world_data[
    world_data["country_ISO"].isin(
        [
            country_iso
            for country_iso in list(
                ms_trial_data["country_ISO"].drop_duplicates()
            )
        ]
    )
][["continent", "region_UN", "subregion"]].drop_duplicates()

In [49]:
[
    continent
    for continent in list(world_data["continent"].drop_duplicates())
    if continent not in list(covered_regions["continent"])
]

['Seven seas (open ocean)', 'Antarctica']

In [50]:
[
    region
    for region in list(world_data["region_UN"].drop_duplicates())
    if region not in list(covered_regions["region_UN"])
]

['Antarctica']

In [51]:
[
    subregion
    for subregion in list(world_data["subregion"].drop_duplicates())
    if subregion not in list(covered_regions["subregion"])
]

['Central Asia',
 'Middle Africa',
 'Seven seas (open ocean)',
 'Western Africa',
 'Antarctica']

### How many people in *Antarctica* and *Seven seas*? Can we drop these data?

In [52]:
world_data[world_data["continent"].isin(["Antarctica", "Seven seas (open ocean)"])]

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group
23,Fr. S. Antarctic Lands,ATF,Seven seas (open ocean),Africa,Seven seas (open ocean),140.0,6. Developing region,2. High income: nonOECD
159,Antarctica,ATA,Antarctica,Antarctica,Antarctica,4490.0,6. Developing region,2. High income: nonOECD


In [53]:
world_data[world_data["continent"].isin(["Antarctica", "Seven seas (open ocean)"])][
    "population_estimate"
].sum() / world_data[world_data["economy_level"] == "6. Developing region"][
    "population_estimate"
].sum()

6.553687575773739e-06

In [54]:
world_data[world_data["continent"].isin(["Antarctica", "Seven seas (open ocean)"])][
    "population_estimate"
].sum() / world_data[world_data["income_group"] == "2. High income: nonOECD"][
    "population_estimate"
].sum()

4.741874038394288e-05

Very few people, and no clinical trial data; dropping these will affect the normalization per capita per economy level and income group only very slighly (10^-6 and 10^-5, respectively).

### Remove *Antarctica* and *Seven seas (open ocean)*

In [55]:
world_data = world_data[
    ~world_data["continent"].isin(["Antarctica", "Seven seas (open ocean)"])
].copy()

### Add consolidated income and development levels

#### Income
We will not distinguish between OECD and non-OECD in the high income group.

In [56]:
consolidated_income_groups = world_data[["income_group"]].drop_duplicates().sort_values("income_group").reset_index(drop=True).copy()

In [57]:
consolidated_income_groups

,income_group
0,1. High income: OECD
1,2. High income: nonOECD
2,3. Upper middle income
3,4. Lower middle income
4,5. Low income


In [58]:
consolidated_income_groups.at[0, "consolidated_income_group"] = "1. High income"
consolidated_income_groups.at[1, "consolidated_income_group"] = "1. High income"
consolidated_income_groups.at[2, "consolidated_income_group"] = "2. Upper middle income"
consolidated_income_groups.at[3, "consolidated_income_group"] = "3. Lower middle income"
consolidated_income_groups.at[4, "consolidated_income_group"] = "4. Low income"

In [59]:
consolidated_income_groups

,income_group,consolidated_income_group
0,1. High income: OECD,1. High income
1,2. High income: nonOECD,1. High income
2,3. Upper middle income,2. Upper middle income
3,4. Lower middle income,3. Lower middle income
4,5. Low income,4. Low income


In [60]:
world_data = pd.merge(left=world_data, right=consolidated_income_groups, on="income_group", how="left")

In [61]:
world_data[world_data["consolidated_income_group"].isna()]

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group,consolidated_income_group


#### Development
We will consolidate G7 and non-G7 for the developed regions, and BRIC, MIKT, and G20 for emerging regions.

In [62]:
consolidated_economy_levels = world_data[["economy_level"]].drop_duplicates().sort_values("economy_level").reset_index(drop=True).copy()

In [63]:
consolidated_economy_levels

,economy_level
0,1. Developed region: G7
1,2. Developed region: nonG7
2,3. Emerging region: BRIC
3,4. Emerging region: MIKT
4,5. Emerging region: G20
5,6. Developing region
6,7. Least developed region


In [64]:
consolidated_economy_levels.at[0, "consolidated_economy_level"] = "1. Developed region"
consolidated_economy_levels.at[1, "consolidated_economy_level"] = "1. Developed region"
consolidated_economy_levels.at[2, "consolidated_economy_level"] = "2. Emerging region"
consolidated_economy_levels.at[3, "consolidated_economy_level"] = "2. Emerging region"
consolidated_economy_levels.at[4, "consolidated_economy_level"] = "2. Emerging region"
consolidated_economy_levels.at[5, "consolidated_economy_level"] = "3. Developing region"
consolidated_economy_levels.at[6, "consolidated_economy_level"] = "4. Least developed region"

In [65]:
consolidated_economy_levels

,economy_level,consolidated_economy_level
0,1. Developed region: G7,1. Developed region
1,2. Developed region: nonG7,1. Developed region
2,3. Emerging region: BRIC,2. Emerging region
3,4. Emerging region: MIKT,2. Emerging region
4,5. Emerging region: G20,2. Emerging region
5,6. Developing region,3. Developing region
6,7. Least developed region,4. Least developed region


In [66]:
world_data = pd.merge(left=world_data, right=consolidated_economy_levels, on="economy_level", how="left")

In [67]:
world_data[world_data["consolidated_economy_level"].isna()]

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group,consolidated_income_group,consolidated_economy_level


## Human Development Index (HDI) data

### HDI categories from statistical annex
Source: HDR21-22_Statistical_Annex_HDI_Table.xlsx
* Very high human development: >= 0.800
* High human development: [0.700, 0.800)
* Medium human development: [0.550, 0.700)
* Low human development: < 0.550

In [68]:
def get_hdi_category(hdi):
    if hdi >= 0.800:
        category = "1. Very high human development"
    elif (hdi >= 0.700) and (hdi < 0.800):
        category = "2. High human development"
    elif (hdi >= 0.550) and (hdi < 0.700):
        category = "3. Medium human development"
    elif hdi < 0.550:
        category = "4. Low human development"
    return category

### HDI data

In [69]:
hdi_raw = pd.read_csv("data/source/unstats/human-development-index.csv")

In [70]:
hdi_raw.head()

,Entity,Code,Year,Human Development Index
0,Afghanistan,AFG,1990,0.273
1,Afghanistan,AFG,1991,0.279
2,Afghanistan,AFG,1992,0.287
3,Afghanistan,AFG,1993,0.297
4,Afghanistan,AFG,1994,0.292


#### Get most recent per country
For simplicity, we will not map on trial start year but instead use the latest available HDI data for each country.

In [71]:
max_year_hdi_per_country = (
    hdi_raw[["Entity", "Code", "Year"]]
    .groupby(["Entity", "Code"])
    .max()
    .reset_index()
)

In [72]:
max_year_hdi_per_country.head()

,Entity,Code,Year
0,Afghanistan,AFG,2021
1,Albania,ALB,2021
2,Algeria,DZA,2021
3,Andorra,AND,2021
4,Angola,AGO,2021


In [73]:
hdi_raw = pd.merge(
    left=hdi_raw,
    right=max_year_hdi_per_country,
    on=["Entity", "Code", "Year"],
    how="inner",
)

In [74]:
hdi_raw

,Entity,Code,Year,Human Development Index
0,Afghanistan,AFG,2021,0.478
1,Albania,ALB,2021,0.796
2,Algeria,DZA,2021,0.745
3,Andorra,AND,2021,0.858
4,Angola,AGO,2021,0.586
...,...,...,...,...
187,Vietnam,VNM,2021,0.703
188,World,OWID_WRL,2021,0.732
189,Yemen,YEM,2021,0.455
190,Zambia,ZMB,2021,0.565


### Rename columns for convenience

In [75]:
hdi_data = (
    hdi_raw[["Entity", "Code", "Human Development Index"]]
    .rename(
        columns={
            "Entity": "hdi_country",
            "Code": "country_ISO",
            "Human Development Index": "hdi",
        }
    )
    .copy()
)

In [76]:
hdi_data.head()

,hdi_country,country_ISO,hdi
0,Afghanistan,AFG,0.478
1,Albania,ALB,0.796
2,Algeria,DZA,0.745
3,Andorra,AND,0.858
4,Angola,AGO,0.586


### HDI available for all countries with trials?

#### Any mismatches?

In [77]:
trial_hdi_mapped_countries = pd.merge(
    left=ms_trial_data[["country_norm", "country_ISO"]].drop_duplicates(),
    right=hdi_data[["hdi_country", "country_ISO"]].drop_duplicates(),
    on="country_ISO",
    how="left",
)

In [78]:
trial_hdi_mapped_countries[trial_hdi_mapped_countries["country_norm"] != trial_hdi_mapped_countries["hdi_country"]]

,country_norm,country_ISO,hdi_country
2,Czech Republic,CZE,Czechia
31,Puerto Rico,PRI,NaN
53,Republic of Korea,KOR,South Korea
64,Taiwan,TWN,NaN
72,New Caledonia,NCL,NaN


In [79]:
not_available = [
    country_ISO
    for country_ISO in list(ms_trial_data["country_ISO"].drop_duplicates())
    if country_ISO not in list(hdi_data["country_ISO"].drop_duplicates())
]

In [80]:
missing_hdi_trials = ms_trial_data[
    ms_trial_data["country_ISO"].isin(not_available)
][["country_norm", "country_ISO"]].drop_duplicates()

In [81]:
missing_hdi_trials

,country_norm,country_ISO
55,Puerto Rico,PRI
927,Taiwan,TWN
1394,New Caledonia,NCL


New Caledonia (France) and Puerto Rico (USA) are not sovereign states, but since they are geographically separated from the state they belong to, we treat them as such.

### HDI available for all countries in the Natural Earth set?

#### Check mapping

In [82]:
world_hdi_country_map = pd.merge(left=world_data[["country_name", "country_ISO"]], right=hdi_data[["hdi_country", "country_ISO"]], on="country_ISO", how="left")

##### Country names - mismatches?

In [83]:
world_hdi_country_map[(world_hdi_country_map["country_name"] != world_hdi_country_map["hdi_country"]) & (~world_hdi_country_map["hdi_country"].isna())]

,country_name,country_ISO,hdi_country
4,United States of America,USA,United States
11,Dem. Rep. Congo,COD,Democratic Republic of Congo
17,Dominican Rep.,DOM,Dominican Republic
23,Timor-Leste,TLS,East Timor
59,Côte d'Ivoire,CIV,Cote d'Ivoire
65,Central African Rep.,CAF,Central African Republic
68,Eq. Guinea,GNQ,Equatorial Guinea
72,eSwatini,SWZ,Eswatini
134,Solomon Is.,SLB,Solomon Islands
168,Bosnia and Herz.,BIH,Bosnia and Herzegovina


#### Missing HDI data

In [84]:
missing_hdi_world = (
    world_hdi_country_map[world_hdi_country_map["hdi_country"].isna()]
    .sort_values("country_name")
    .reset_index(drop=True)[["country_name", "country_ISO"]]
    .copy()
)

In [85]:
missing_hdi_world

,country_name,country_ISO
0,Falkland Is.,FLK
1,Greenland,GRL
2,Kosovo,KOS
3,N. Cyprus,CYN
4,New Caledonia,NCL
5,North Korea,PRK
6,Palestine,PSX
7,Puerto Rico,PRI
8,S. Sudan,SDS
9,Somalia,SOM


### Combine missing HDI, then collect data manually
We create a list of countries for which HDI data is missing, then collect the data manually.

In [86]:
missing_hdi = (
    pd.concat(
        [
            missing_hdi_trials.rename(columns={"country_norm": "country_name"}),
            missing_hdi_world,
        ]
    )
    .drop_duplicates()
    .sort_values("country_name")
    .reset_index(drop=True)
)

In [87]:
missing_hdi

,country_name,country_ISO
0,Falkland Is.,FLK
1,Greenland,GRL
2,Kosovo,KOS
3,N. Cyprus,CYN
4,New Caledonia,NCL
5,North Korea,PRK
6,Palestine,PSX
7,Puerto Rico,PRI
8,S. Sudan,SDS
9,Somalia,SOM


In [ ]:
# safety switch
polse

In [ ]:
missing_hdi.to_csv("data/manual/socioeconomic/manual_hdi_export.csv")

### Import manually compiled HDI data

In [89]:
manual_hdi = pd.read_csv("data/manual/socioeconomic/manual_hdi.csv")

In [90]:
manual_hdi

,country_name,country_ISO,hdi,source
0,Falkland Is.,FLK,0.874,https://www.indexmundi.com/falkland_islands_(i...
1,Greenland,GRL,0.786,https://www.indexmundi.com/greenland/
2,Kosovo,KOS,0.759,https://globaldatalab.org/shdi/shdi/XKO/?level...
3,N. Cyprus,CYN,0.887,https://www.indexmundi.com/cyprus/
4,New Caledonia,NCL,0.813,https://www.indexmundi.com/new_caledonia/
5,North Korea,PRK,0.766,https://en.wikipedia.org/wiki/International_ra...
6,Palestine,PSX,0.716,https://hdr.undp.org/data-center/human-develop...
7,Puerto Rico,PRI,0.845,https://www.indexmundi.com/puerto_rico/
8,S. Sudan,SDS,0.385,https://hdr.undp.org/data-center/human-develop...
9,Somalia,SOM,0.361,https://www.indexmundi.com/somalia/


We are only looking at HDI categories (no regression etc.), so we fill the missing values with zeros as those regions are most likely in the lowest category. **Important:** These countries must not be used in any kind of regression analyses!

In [91]:
manual_hdi = manual_hdi.fillna(0)

In [92]:
manual_hdi

,country_name,country_ISO,hdi,source
0,Falkland Is.,FLK,0.874,https://www.indexmundi.com/falkland_islands_(i...
1,Greenland,GRL,0.786,https://www.indexmundi.com/greenland/
2,Kosovo,KOS,0.759,https://globaldatalab.org/shdi/shdi/XKO/?level...
3,N. Cyprus,CYN,0.887,https://www.indexmundi.com/cyprus/
4,New Caledonia,NCL,0.813,https://www.indexmundi.com/new_caledonia/
5,North Korea,PRK,0.766,https://en.wikipedia.org/wiki/International_ra...
6,Palestine,PSX,0.716,https://hdr.undp.org/data-center/human-develop...
7,Puerto Rico,PRI,0.845,https://www.indexmundi.com/puerto_rico/
8,S. Sudan,SDS,0.385,https://hdr.undp.org/data-center/human-develop...
9,Somalia,SOM,0.361,https://www.indexmundi.com/somalia/


### Add manually compiled data to HDI set

In [93]:
hdi_data = pd.concat([hdi_data, manual_hdi[["country_name", "country_ISO", "hdi"]].rename(columns={"country_name": "hdi_country"})]).reset_index(drop=True)

In [94]:
hdi_data

,hdi_country,country_ISO,hdi
0,Afghanistan,AFG,0.478
1,Albania,ALB,0.796
2,Algeria,DZA,0.745
3,Andorra,AND,0.858
4,Angola,AGO,0.586
...,...,...,...
200,S. Sudan,SDS,0.385
201,Somalia,SOM,0.361
202,Somaliland,SOL,0.000
203,Taiwan,TWN,0.916


### Add categories

In [95]:
hdi_data["hdi_category"] = hdi_data.apply(lambda row: get_hdi_category(row["hdi"]), axis=1)

In [96]:
hdi_data.head()

,hdi_country,country_ISO,hdi,hdi_category
0,Afghanistan,AFG,0.478,4. Low human development
1,Albania,ALB,0.796,2. High human development
2,Algeria,DZA,0.745,2. High human development
3,Andorra,AND,0.858,1. Very high human development
4,Angola,AGO,0.586,3. Medium human development


# Part 3 - trial and site counts

#### Create dataframes with trial and site counts
Trials and sites per:
* Continent
* Region
* Country
* HDI category
* Income category
* Economical development category

Normalized data:
* Data per capita
* Normalize over phases or groups
* Compute expected sites per capita and difference to actual number (factor, log of factor)

## Create combined dataframe

In [97]:
trials_socioeconomic_data = pd.merge(left=ms_trial_data, right=world_data, on="country_ISO", how="left")

In [98]:
trials_socioeconomic_data[trials_socioeconomic_data["population_estimate"].isna()]

,nct_number,study_status,sponsor,phase,enrollment,primary_purpose,start_year,primary_completion_year,completion_year,country_norm,...,n_sites,country_name,continent,region_UN,subregion,population_estimate,economy_level,income_group,consolidated_income_group,consolidated_economy_level


In [99]:
trials_socioeconomic_data = pd.merge(left=trials_socioeconomic_data, right=hdi_data, on="country_ISO", how="left")

In [100]:
trials_socioeconomic_data[trials_socioeconomic_data["hdi"].isna()]

,nct_number,study_status,sponsor,phase,enrollment,primary_purpose,start_year,primary_completion_year,completion_year,country_norm,...,region_UN,subregion,population_estimate,economy_level,income_group,consolidated_income_group,consolidated_economy_level,hdi_country,hdi,hdi_category


## Counts

### Population numbers
If we want to normalize per capita, we need population numbers on country, continent, region, HDI category, income group, and development category level.

In [101]:
world_data.head()

,country_name,country_ISO,continent,region_UN,subregion,population_estimate,economy_level,income_group,consolidated_income_group,consolidated_economy_level
0,Fiji,FJI,Oceania,Oceania,Melanesia,889953.0,6. Developing region,4. Lower middle income,3. Lower middle income,3. Developing region
1,Tanzania,TZA,Africa,Africa,Eastern Africa,58005463.0,7. Least developed region,5. Low income,4. Low income,4. Least developed region
2,W. Sahara,SAH,Africa,Africa,Northern Africa,603253.0,7. Least developed region,5. Low income,4. Low income,4. Least developed region
3,Canada,CAN,North America,Americas,Northern America,37589262.0,1. Developed region: G7,1. High income: OECD,1. High income,1. Developed region
4,United States of America,USA,North America,Americas,Northern America,328239523.0,1. Developed region: G7,1. High income: OECD,1. High income,1. Developed region


In [102]:
len(world_data["country_ISO"]), len(world_data["country_ISO"].drop_duplicates())

(176, 176)

In [103]:
world_data_count_base = (
    world_data[
        [
            "country_ISO",
            "continent",
            "subregion",
            "economy_level",
            "consolidated_economy_level",
            "income_group",
            "consolidated_income_group",
            "population_estimate",
        ]
    ]
    .rename(columns={"continent": "country_continent"})
    .copy()
)

In [104]:
world_data_count_base = pd.merge(
    left=world_data_count_base,
    right=hdi_data[["country_ISO", "hdi_category"]],
    on="country_ISO",
    how="left",
)

In [105]:
world_data_count_base[world_data_count_base["hdi_category"].isna()]

,country_ISO,country_continent,subregion,economy_level,consolidated_economy_level,income_group,consolidated_income_group,population_estimate,hdi_category


In [106]:
len(world_data_count_base["country_ISO"]), len(world_data_count_base["country_ISO"].drop_duplicates())

(176, 176)

In [107]:
grouper_column_names = [
    "country_ISO",
    "country_continent",
    "subregion",
    "economy_level",
    "consolidated_economy_level",
    "income_group",
    "consolidated_income_group",
    "hdi_category",
]

In [108]:
population_data = {
    grouper_column_name: counters.sum_over_groupby(
        data=world_data_count_base,
        sum_column_name="population_estimate",
        groupby_column_names=[grouper_column_name],
    )
    for grouper_column_name in grouper_column_names
}

In [109]:
population_data["country_continent"]

,country_continent,population_estimate
0,Africa,1.306370e+09
1,Asia,4.555914e+09
2,Europe,7.454125e+08
3,North America,5.837560e+08
4,Oceania,4.120487e+07
5,South America,4.270667e+08


Sanity check - there is no double counting in population per group, so the totals should be equal.

In [110]:
np.unique([population_data[grouper_column_name]["population_estimate"].sum() for grouper_column_name in grouper_column_names])

array([7.65972439e+09])

### Trial counts

#### Overall

In [111]:
trial_counts = {
    grouper_column_name: counters.count_trials(
        trial_data=trials_socioeconomic_data,
        groupby_column_names=[grouper_column_name],
        trial_id_column_name="nct_number",
    )
    for grouper_column_name in grouper_column_names
}

In [112]:
trial_counts["country_continent"]

,country_continent,n_trials
0,Africa,29
1,Asia,140
2,Europe,297
3,North America,267
4,Oceania,76
5,South America,54


##### Make sure groups with 0 trials are included, too

In [113]:
for grouper_column_name in grouper_column_names:
    trial_counts[grouper_column_name] = pd.merge(
        left=trial_counts[grouper_column_name],
        right=population_data[grouper_column_name][[grouper_column_name]],
        on=grouper_column_name,
        how="outer",
    ).fillna(0)

#### Per trial phase

In [114]:
trial_counts_per_phase = {
    grouper_column_name: counters.count_trials(
        trial_data=trials_socioeconomic_data,
        groupby_column_names=["phase", grouper_column_name],
        trial_id_column_name="nct_number",
    )
    for grouper_column_name in grouper_column_names
}

In [115]:
trial_counts_per_phase["country_continent"].head()

,phase,country_continent,n_trials
0,PHASE1,Asia,6
1,PHASE1,Europe,27
2,PHASE1,North America,25
3,PHASE1,Oceania,12
4,PHASE1,South America,1


##### Make sure groups with 0 trials are included, too

In [116]:
for grouper_column_name in grouper_column_names:
    trial_counts_per_phase[grouper_column_name] = pd.merge(
        left=trial_counts_per_phase[grouper_column_name],
        right=pd.merge(
            left=population_data[grouper_column_name][[grouper_column_name]],
            right=trials_socioeconomic_data[["phase"]].drop_duplicates(),
            how="cross",
        ),
        on=[grouper_column_name, "phase"],
        how="outer",
    ).fillna(0)

In [117]:
trial_counts_per_phase["income_group"].sort_values("income_group", ascending=False).head()

,phase,income_group,n_trials
19,PHASE1,5. Low income,0.0
18,PHASE2,5. Low income,0.0
17,PHASE3,5. Low income,0.0
15,PHASE4,5. Low income,1.0
6,PHASE2,4. Lower middle income,32.0


### Site counts

#### Overall

In [118]:
site_counts = {
    grouper_column_name: counters.sum_over_groupby(
        data=trials_socioeconomic_data,
        sum_column_name="n_sites",
        groupby_column_names=[grouper_column_name],
    )
    for grouper_column_name in grouper_column_names
}

In [119]:
site_counts["country_continent"]

,country_continent,n_sites
0,Africa,107.0
1,Asia,1423.0
2,Europe,12716.0
3,North America,6322.0
4,Oceania,313.0
5,South America,431.0


##### Make sure groups with 0 trials are included, too

In [120]:
for grouper_column_name in grouper_column_names:
    site_counts[grouper_column_name] = pd.merge(
        left=site_counts[grouper_column_name],
        right=population_data[grouper_column_name][[grouper_column_name]],
        on=grouper_column_name,
        how="outer",
    ).fillna(0)

In [121]:
site_counts["economy_level"].sort_values("economy_level", ascending=False).head()

,economy_level,n_sites
6,7. Least developed region,0.0
5,6. Developing region,1469.0
4,5. Emerging region: G20,318.0
3,4. Emerging region: MIKT,565.0
2,3. Emerging region: BRIC,1691.0


Sanity check - there is no double counting in sites, so the totals should be equal.

In [122]:
np.unique([site_counts[grouper_column_name]["n_sites"].sum() for grouper_column_name in grouper_column_names])

array([21312.])

#### Per trial phase

In [123]:
site_counts_per_phase = {
    grouper_column_name: counters.sum_over_groupby(
        data=trials_socioeconomic_data,
        sum_column_name="n_sites",
        groupby_column_names=["phase", grouper_column_name],
    )
    for grouper_column_name in grouper_column_names
}

In [124]:
site_counts_per_phase["country_continent"].head()

,phase,country_continent,n_sites
0,PHASE1,Asia,9.0
1,PHASE1,Europe,63.0
2,PHASE1,North America,187.0
3,PHASE1,Oceania,17.0
4,PHASE1,South America,1.0


##### Make sure groups with 0 trials are included, too

In [125]:
for grouper_column_name in grouper_column_names:
    site_counts_per_phase[grouper_column_name] = pd.merge(
        left=site_counts_per_phase[grouper_column_name],
        right=pd.merge(
            left=population_data[grouper_column_name][[grouper_column_name]],
            right=trials_socioeconomic_data[["phase"]].drop_duplicates(),
            how="cross",
        ),
        on=[grouper_column_name, "phase"],
        how="outer",
    ).fillna(0)

In [126]:
site_counts_per_phase["income_group"].sort_values("income_group", ascending=False).head()

,phase,income_group,n_sites
19,PHASE1,5. Low income,0.0
18,PHASE2,5. Low income,0.0
17,PHASE3,5. Low income,0.0
15,PHASE4,5. Low income,1.0
6,PHASE2,4. Lower middle income,203.0


In [127]:
np.unique([site_counts_per_phase[grouper_column_name]["n_sites"].sum() for grouper_column_name in grouper_column_names])

array([21312.])

## Normalized counts

### Trials

#### Overall

In [128]:
trial_normalization = len(trials_socioeconomic_data["nct_number"].drop_duplicates())

In [129]:
for grouper_column_name in grouper_column_names:
    trial_counts[grouper_column_name]["n_trials_overall"] = trial_normalization
    trial_counts[grouper_column_name]["of_total_trials_overall"] = trial_counts[grouper_column_name]["n_trials"] / trial_normalization


In [130]:
trial_counts["consolidated_income_group"]

,consolidated_income_group,n_trials,n_trials_overall,of_total_trials_overall
0,1. High income,414,435,0.951724
1,2. Upper middle income,171,435,0.393103
2,3. Lower middle income,102,435,0.234483
3,4. Low income,1,435,0.002299


In [131]:
trial_counts["hdi_category"]

,hdi_category,n_trials,n_trials_overall,of_total_trials_overall
0,1. Very high human development,424.0,435,0.974713
1,2. High human development,136.0,435,0.312644
2,3. Medium human development,34.0,435,0.078161
3,4. Low human development,0.0,435,0.000000


#### Per trial phase

In [132]:
trial_normalization_per_phase = counters.count_trials(
    trial_data=trials_socioeconomic_data, groupby_column_names=["phase"], trial_id_column_name="nct_number"
).rename(columns={"n_trials": "n_trials_phase"})

In [133]:
trial_normalization_per_phase

,phase,n_trials_phase
0,PHASE1,64
1,PHASE2,123
2,PHASE3,183
3,PHASE4,79


In [134]:
for grouper_column_name in grouper_column_names:
    trial_counts_per_phase[grouper_column_name] = pd.merge(
        left=trial_counts_per_phase[grouper_column_name],
        right=trial_normalization_per_phase,
        on="phase",
    )
    trial_counts_per_phase[grouper_column_name]["of_total_trials_phase"] = (
        trial_counts_per_phase[grouper_column_name]["n_trials"]
        / trial_counts_per_phase[grouper_column_name]["n_trials_phase"]
    )

In [135]:
trial_counts_per_phase["consolidated_income_group"]

,phase,consolidated_income_group,n_trials,n_trials_phase,of_total_trials_phase
0,PHASE1,1. High income,59.0,64,0.921875
1,PHASE1,2. Upper middle income,4.0,64,0.062500
2,PHASE1,3. Lower middle income,2.0,64,0.031250
3,PHASE1,4. Low income,0.0,64,0.000000
4,PHASE2,1. High income,118.0,123,0.959350
5,PHASE2,2. Upper middle income,56.0,123,0.455285
6,PHASE2,3. Lower middle income,32.0,123,0.260163
7,PHASE2,4. Low income,0.0,123,0.000000
8,PHASE3,1. High income,176.0,183,0.961749
9,PHASE3,2. Upper middle income,104.0,183,0.568306


### Trial sites

#### Overall

In [136]:
site_normalization = trials_socioeconomic_data["n_sites"].sum()

In [137]:
for grouper_column_name in grouper_column_names:
    site_counts[grouper_column_name]["n_sites_overall"] = site_normalization
    site_counts[grouper_column_name]["of_total_sites_overall"] = site_counts[grouper_column_name]["n_sites"] / site_normalization


In [138]:
site_counts["consolidated_income_group"]

,consolidated_income_group,n_sites,n_sites_overall,of_total_sites_overall
0,1. High income,16507.0,21312.0,0.774540
1,2. Upper middle income,3615.0,21312.0,0.169623
2,3. Lower middle income,1189.0,21312.0,0.055790
3,4. Low income,1.0,21312.0,0.000047


Sanity check: fractions should add up to 1.

In [139]:
[site_counts[grouper_column_name]["of_total_sites_overall"].sum() for grouper_column_name in grouper_column_names]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999999999999999]

#### Per trial phase

In [140]:
site_normalization_per_phase = counters.sum_over_groupby(
    data=trials_socioeconomic_data,
    sum_column_name="n_sites",
    groupby_column_names=["phase"],
).rename(columns={"n_sites": "n_sites_phase"})

In [141]:
site_normalization_per_phase

,phase,n_sites_phase
0,PHASE1,277.0
1,PHASE2,3432.0
2,PHASE3,15036.0
3,PHASE4,2567.0


In [142]:
for grouper_column_name in grouper_column_names:
    site_counts_per_phase[grouper_column_name] = pd.merge(
        left=site_counts_per_phase[grouper_column_name],
        right=site_normalization_per_phase,
        on="phase",
    )
    site_counts_per_phase[grouper_column_name]["of_total_sites_phase"] = (
        site_counts_per_phase[grouper_column_name]["n_sites"]
        / site_counts_per_phase[grouper_column_name]["n_sites_phase"]
    )

In [143]:
site_counts_per_phase["consolidated_income_group"]

,phase,consolidated_income_group,n_sites,n_sites_phase,of_total_sites_phase
0,PHASE1,1. High income,270.0,277.0,0.974729
1,PHASE1,2. Upper middle income,4.0,277.0,0.014440
2,PHASE1,3. Lower middle income,3.0,277.0,0.010830
3,PHASE1,4. Low income,0.0,277.0,0.000000
4,PHASE2,1. High income,2752.0,3432.0,0.801865
5,PHASE2,2. Upper middle income,477.0,3432.0,0.138986
6,PHASE2,3. Lower middle income,203.0,3432.0,0.059149
7,PHASE2,4. Low income,0.0,3432.0,0.000000
8,PHASE3,1. High income,11067.0,15036.0,0.736034
9,PHASE3,2. Upper middle income,3008.0,15036.0,0.200053


Sanity check: the fractions should add up to 1 for each phase, thus all totals should be 4 (with some tiny rounding errors, thus 'isclose').

In [144]:
[np.isclose(4, site_counts_per_phase[grouper_column_name]["of_total_sites_phase"].sum()) for grouper_column_name in grouper_column_names]

[True, True, True, True, True, True, True, True]

## Per capita counts

### Trials

#### Overall

In [145]:
for grouper_column_name in grouper_column_names:
    updated_counts_df = trial_counts[grouper_column_name]
    updated_counts_df = pd.merge(
        left=updated_counts_df,
        right=population_data[grouper_column_name],
        on=grouper_column_name,
    )
    updated_counts_df["trials_per_capita"] = (
        updated_counts_df["n_trials"] / updated_counts_df["population_estimate"]
    )
    updated_counts_df["n_trials_expected"] = (
        updated_counts_df["population_estimate"]
        * trial_normalization
        / population_data[grouper_column_name]["population_estimate"].sum()
    )
    updated_counts_df["factor_deviation_n_trials_from_expected"] = (
        updated_counts_df["n_trials"] / updated_counts_df["n_trials_expected"]
    )
    trial_counts[grouper_column_name] = updated_counts_df

In [146]:
trial_counts["consolidated_income_group"]

,consolidated_income_group,n_trials,n_trials_overall,of_total_trials_overall,population_estimate,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected
0,1. High income,414,435,0.951724,1.172607e+09,3.530595e-07,66.592987,6.216871
1,2. Upper middle income,171,435,0.393103,2.643713e+09,6.468175e-08,150.137936,1.138953
2,3. Lower middle income,102,435,0.234483,2.855863e+09,3.571600e-08,162.186024,0.628907
3,4. Low income,1,435,0.002299,9.875419e+08,1.012615e-09,56.083053,0.017831


#### Per trial phase

In [147]:
for grouper_column_name in grouper_column_names:
    updated_counts_df = trial_counts_per_phase[grouper_column_name]
    updated_counts_df = pd.merge(
        left=updated_counts_df,
        right=population_data[grouper_column_name],
        on=grouper_column_name,
    )
    updated_counts_df["trials_per_capita"] = (
        updated_counts_df["n_trials"] / updated_counts_df["population_estimate"]
    )
    updated_counts_df["n_trials_expected"] = (
        updated_counts_df["population_estimate"]
        * updated_counts_df["n_trials_phase"]
        / population_data[grouper_column_name]["population_estimate"].sum()
    )
    updated_counts_df["factor_deviation_n_trials_from_expected"] = (
        updated_counts_df["n_trials"] / updated_counts_df["n_trials_expected"]
    )
    trial_counts_per_phase[grouper_column_name] = updated_counts_df

In [148]:
trial_counts_per_phase["consolidated_income_group"]

,phase,consolidated_income_group,n_trials,n_trials_phase,of_total_trials_phase,population_estimate,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected
0,PHASE1,1. High income,59.0,64,0.921875,1.172607e+09,5.031525e-08,9.797589,6.021890
1,PHASE2,1. High income,118.0,123,0.959350,1.172607e+09,1.006305e-07,18.829741,6.266682
2,PHASE3,1. High income,176.0,183,0.961749,1.172607e+09,1.500929e-07,28.014981,6.282353
3,PHASE4,1. High income,72.0,79,0.911392,1.172607e+09,6.140166e-08,12.093899,5.953415
4,PHASE1,2. Upper middle income,4.0,64,0.062500,2.643713e+09,1.513023e-09,22.089260,0.181083
5,PHASE2,2. Upper middle income,56.0,123,0.455285,2.643713e+09,2.118233e-08,42.452796,1.319112
6,PHASE3,2. Upper middle income,104.0,183,0.568306,2.643713e+09,3.933861e-08,63.161477,1.646573
7,PHASE4,2. Upper middle income,13.0,79,0.164557,2.643713e+09,4.917326e-09,27.266430,0.476777
8,PHASE1,3. Lower middle income,2.0,64,0.031250,2.855863e+09,7.003138e-10,23.861852,0.083816
9,PHASE2,3. Lower middle income,32.0,123,0.260163,2.855863e+09,1.120502e-08,45.859496,0.697784


### Trial sites

#### Overall

In [149]:
site_counts["consolidated_income_group"]

,consolidated_income_group,n_sites,n_sites_overall,of_total_sites_overall
0,1. High income,16507.0,21312.0,0.774540
1,2. Upper middle income,3615.0,21312.0,0.169623
2,3. Lower middle income,1189.0,21312.0,0.055790
3,4. Low income,1.0,21312.0,0.000047


In [150]:
for grouper_column_name in grouper_column_names:
    updated_counts_df = site_counts[grouper_column_name]
    updated_counts_df = pd.merge(
        left=updated_counts_df,
        right=population_data[grouper_column_name],
        on=grouper_column_name,
    )
    updated_counts_df["sites_per_capita"] = (
        updated_counts_df["n_sites"] / updated_counts_df["population_estimate"]
    )
    updated_counts_df["n_sites_expected"] = (
        updated_counts_df["population_estimate"]
        * updated_counts_df["n_sites_overall"]
        / population_data[grouper_column_name]["population_estimate"].sum()
    )
    updated_counts_df["factor_deviation_n_sites_from_expected"] = (
        updated_counts_df["n_sites"] / updated_counts_df["n_sites_expected"]
    )
    site_counts[grouper_column_name] = updated_counts_df

In [151]:
site_counts["consolidated_income_group"]

,consolidated_income_group,n_sites,n_sites_overall,of_total_sites_overall,population_estimate,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,1. High income,16507.0,21312.0,0.774540,1.172607e+09,1.407718e-05,3262.597109,5.059466
1,2. Upper middle income,3615.0,21312.0,0.169623,2.643713e+09,1.367395e-06,7355.723437,0.491454
2,3. Lower middle income,1189.0,21312.0,0.055790,2.855863e+09,4.163366e-07,7945.996658,0.149635
3,4. Low income,1.0,21312.0,0.000047,9.875419e+08,1.012615e-09,2747.682797,0.000364


In [152]:
site_counts["country_continent"]

,country_continent,n_sites,n_sites_overall,of_total_sites_overall,population_estimate,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,Africa,107.0,21312.0,0.005021,1.306370e+09,8.190634e-08,3634.773343,0.029438
1,Asia,1423.0,21312.0,0.066770,4.555914e+09,3.123413e-07,12676.127426,0.112258
2,Europe,12716.0,21312.0,0.596659,7.454125e+08,1.705901e-05,2073.995012,6.131162
3,North America,6322.0,21312.0,0.296640,5.837560e+08,1.082987e-05,1624.211003,3.892351
4,Oceania,313.0,21312.0,0.014687,4.120487e+07,7.596189e-06,114.646197,2.730139
5,South America,431.0,21312.0,0.020223,4.270667e+08,1.009210e-06,1188.247020,0.362719


Sanity check: expected sites should sum up to actual sites.

In [153]:
[np.isclose(21312, site_counts[grouper_column_name]["n_sites_expected"].sum()) for grouper_column_name in grouper_column_names]

[True, True, True, True, True, True, True, True]

#### Per trial phase

In [154]:
for grouper_column_name in grouper_column_names:
    updated_counts_df = site_counts_per_phase[grouper_column_name]
    updated_counts_df = pd.merge(
        left=updated_counts_df,
        right=population_data[grouper_column_name],
        on=grouper_column_name,
    )
    updated_counts_df["sites_per_capita"] = (
        updated_counts_df["n_sites"] / updated_counts_df["population_estimate"]
    )
    updated_counts_df["n_sites_expected"] = (
        updated_counts_df["population_estimate"]
        * updated_counts_df["n_sites_phase"]
        / population_data[grouper_column_name]["population_estimate"].sum()
    )
    updated_counts_df["factor_deviation_n_sites_from_expected"] = (
        updated_counts_df["n_sites"] / updated_counts_df["n_sites_expected"]
    )
    site_counts_per_phase[grouper_column_name] = updated_counts_df

In [155]:
site_counts_per_phase["consolidated_income_group"]

,phase,consolidated_income_group,n_sites,n_sites_phase,of_total_sites_phase,population_estimate,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,1. High income,270.0,277.0,0.974729,1.172607e+09,2.302562e-07,42.405190,6.367145
1,PHASE2,1. High income,2752.0,3432.0,0.801865,1.172607e+09,2.346908e-06,525.395706,5.237957
2,PHASE3,1. High income,11067.0,15036.0,0.736034,1.172607e+09,9.437947e-06,2301.821046,4.807932
3,PHASE4,1. High income,2418.0,2567.0,0.941956,1.172607e+09,2.062072e-06,392.975168,6.153061
4,PHASE1,2. Upper middle income,4.0,277.0,0.014440,2.643713e+09,1.513023e-09,95.605077,0.041839
5,PHASE2,2. Upper middle income,477.0,3432.0,0.138986,2.643713e+09,1.804280e-07,1184.536544,0.402689
6,PHASE3,2. Upper middle income,3008.0,15036.0,0.200053,2.643713e+09,1.137794e-06,5189.595420,0.579621
7,PHASE4,2. Upper middle income,126.0,2567.0,0.049085,2.643713e+09,4.766024e-08,885.986396,0.142214
8,PHASE1,3. Lower middle income,3.0,277.0,0.010830,2.855863e+09,1.050471e-09,103.277077,0.029048
9,PHASE2,3. Lower middle income,203.0,3432.0,0.059149,2.855863e+09,7.108185e-08,1279.591804,0.158644


In [156]:
[np.isclose(21312, site_counts_per_phase[grouper_column_name]["n_sites_expected"].sum()) for grouper_column_name in grouper_column_names]

[True, True, True, True, True, True, True, True]

## Consolidate datasets

In [157]:
consolidated_counts_overall = {
    grouper_column_name: pd.merge(
        left=trial_counts[grouper_column_name],
        right=site_counts[grouper_column_name].drop(columns=["population_estimate"]),
        on=grouper_column_name,
    )[
        [
            grouper_column_name,
            "population_estimate",
            "n_trials",
            "n_trials_overall",
            "of_total_trials_overall",
            "trials_per_capita",
            "n_trials_expected",
            "factor_deviation_n_trials_from_expected",
            "n_sites",
            "n_sites_overall",
            "of_total_sites_overall",
            "sites_per_capita",
            "n_sites_expected",
            "factor_deviation_n_sites_from_expected",
        ]
    ]
    for grouper_column_name in grouper_column_names
}

In [158]:
for grouper_column_name in grouper_column_names:
    display(consolidated_counts_overall[grouper_column_name].head())

,country_ISO,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,ALB,2854191.0,2.0,435,0.004598,7.007240e-07,0.162091,12.338741,2.0,21312.0,0.000094,7.007240e-07,7.941346,0.251846
1,ARE,9770529.0,2.0,435,0.004598,2.046972e-07,0.554874,3.604423,3.0,21312.0,0.000141,3.070458e-07,27.184988,0.110355
2,ARG,44938712.0,36.0,435,0.082759,8.010911e-07,2.552094,14.106061,153.0,21312.0,0.007179,3.404637e-06,125.035025,1.223657
3,ARM,2957731.0,1.0,435,0.002299,3.380970e-07,0.167971,5.953402,2.0,21312.0,0.000094,6.761940e-07,8.229430,0.243030
4,AUS,25364307.0,72.0,435,0.165517,2.838635e-06,1.440453,49.984273,280.0,21312.0,0.013138,1.103913e-05,70.572266,3.967564


,country_continent,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,Africa,1.306370e+09,29,435,0.066667,2.219891e-08,74.189490,0.390891,107.0,21312.0,0.005021,8.190634e-08,3634.773343,0.029438
1,Asia,4.555914e+09,140,435,0.321839,3.072929e-08,258.732894,0.541099,1423.0,21312.0,0.066770,3.123413e-07,12676.127426,0.112258
2,Europe,7.454125e+08,297,435,0.682759,3.984371e-07,42.332387,7.015905,12716.0,21312.0,0.596659,1.705901e-05,2073.995012,6.131162
3,North America,5.837560e+08,267,435,0.613793,4.573829e-07,33.151829,8.053854,6322.0,21312.0,0.296640,1.082987e-05,1624.211003,3.892351
4,Oceania,4.120487e+07,76,435,0.174713,1.844442e-06,2.340048,32.477971,313.0,21312.0,0.014687,7.596189e-06,114.646197,2.730139


,subregion,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,Australia and New Zealand,3.028131e+07,76.0,435,0.174713,2.509799e-06,1.719692,44.193954,312.0,21312.0,0.014640,1.030339e-05,84.253059,3.703130
1,Caribbean,4.126195e+07,22.0,435,0.050575,5.331789e-07,2.343289,9.388514,23.0,21312.0,0.001079,5.574143e-07,114.804991,0.200340
2,Central America,1.766091e+08,48.0,435,0.110345,2.717867e-07,10.029728,4.785773,206.0,21312.0,0.009666,1.166418e-06,491.387486,0.419221
3,Eastern Africa,4.329612e+08,1.0,435,0.002299,2.309676e-09,24.588108,0.040670,1.0,21312.0,0.000047,2.309676e-09,1204.647729,0.000830
4,Eastern Asia,1.628149e+09,42.0,435,0.096552,2.579617e-08,92.463470,0.454233,531.0,21312.0,0.024916,3.261373e-07,4530.072372,0.117217


,economy_level,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,1. Developed region: G7,7.694182e+08,369.0,435,0.848276,4.795831e-07,43.695687,8.444769,10810.0,21312.0,0.507226,1.404958e-05,2140.787306,5.049544
1,2. Developed region: nonG7,3.524729e+08,242.0,435,0.556322,6.865776e-07,20.017132,12.089644,6459.0,21312.0,0.303069,1.832481e-05,980.701424,6.586102
2,3. Emerging region: BRIC,3.119556e+09,139.0,435,0.319540,4.455763e-08,177.161306,0.784596,1691.0,21312.0,0.079345,5.420644e-07,8679.682212,0.194823
3,4. Emerging region: MIKT,5.333398e+08,88.0,435,0.202299,1.649980e-07,30.288664,2.905377,565.0,21312.0,0.026511,1.059362e-06,1483.935642,0.380744
4,5. Emerging region: G20,1.164130e+09,63.0,435,0.144828,5.411765e-08,66.111615,0.952934,318.0,21312.0,0.014921,2.731653e-07,3239.013183,0.098178


,consolidated_economy_level,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,1. Developed region,1.121891e+09,414.0,435,0.951724,3.690198e-07,63.712819,6.497907,17269.0,21312.0,0.810295,1.539276e-05,3121.488729,5.532296
1,2. Emerging region,4.817026e+09,159.0,435,0.365517,3.300792e-08,273.561585,0.581222,2574.0,21312.0,0.120777,5.343546e-07,13402.631037,0.192052
2,3. Developing region,7.064679e+08,124.0,435,0.285057,1.755211e-07,40.120702,3.090674,1469.0,21312.0,0.068928,2.079359e-06,1965.637708,0.747340
3,4. Least developed region,1.014339e+09,0.0,435,0.000000,0.000000e+00,57.604894,0.000000,0.0,21312.0,0.000000,0.000000e+00,2822.242526,0.000000


,income_group,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,1. High income: OECD,1.074971e+09,413,435,0.949425,3.841965e-07,61.048179,6.765149,16305.0,21312.0,0.765062,1.516786e-05,2990.939766,5.451464
1,2. High income: nonOECD,9.763609e+07,67,435,0.154023,6.862217e-07,5.544808,12.083376,202.0,21312.0,0.009478,2.068907e-06,271.657342,0.743584
2,3. Upper middle income,2.643713e+09,171,435,0.393103,6.468175e-08,150.137936,1.138953,3615.0,21312.0,0.169623,1.367395e-06,7355.723437,0.491454
3,4. Lower middle income,2.855863e+09,102,435,0.234483,3.571600e-08,162.186024,0.628907,1189.0,21312.0,0.055790,4.163366e-07,7945.996658,0.149635
4,5. Low income,9.875419e+08,1,435,0.002299,1.012615e-09,56.083053,0.017831,1.0,21312.0,0.000047,1.012615e-09,2747.682797,0.000364


,consolidated_income_group,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,1. High income,1.172607e+09,414,435,0.951724,3.530595e-07,66.592987,6.216871,16507.0,21312.0,0.774540,1.407718e-05,3262.597109,5.059466
1,2. Upper middle income,2.643713e+09,171,435,0.393103,6.468175e-08,150.137936,1.138953,3615.0,21312.0,0.169623,1.367395e-06,7355.723437,0.491454
2,3. Lower middle income,2.855863e+09,102,435,0.234483,3.571600e-08,162.186024,0.628907,1189.0,21312.0,0.055790,4.163366e-07,7945.996658,0.149635
3,4. Low income,9.875419e+08,1,435,0.002299,1.012615e-09,56.083053,0.017831,1.0,21312.0,0.000047,1.012615e-09,2747.682797,0.000364


,hdi_category,population_estimate,n_trials,n_trials_overall,of_total_trials_overall,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_overall,of_total_sites_overall,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,1. Very high human development,1.640870e+09,424.0,435,0.974713,2.583995e-07,93.185916,4.550044,18964.0,21312.0,0.889827,1.155728e-05,4565.467243,4.153792
1,2. High human development,2.732477e+09,136.0,435,0.312644,4.977169e-08,155.178902,0.876408,2125.0,21312.0,0.099709,7.776826e-07,7602.695985,0.279506
2,3. Medium human development,2.160216e+09,34.0,435,0.078161,1.573917e-08,122.679858,0.277144,223.0,21312.0,0.010464,1.032304e-07,6010.466964,0.037102
3,4. Low human development,1.126161e+09,0.0,435,0.000000,0.000000e+00,63.955324,0.000000,0.0,21312.0,0.000000,0.000000e+00,3133.369808,0.000000


In [159]:
consolidated_counts_per_phase = {
    grouper_column_name: pd.merge(
        left=trial_counts_per_phase[grouper_column_name],
        right=site_counts_per_phase[grouper_column_name].drop(
            columns=["population_estimate"]
        ),
        on=[grouper_column_name, "phase"],
    )[
        [
            "phase",
            grouper_column_name,
            "population_estimate",
            "n_trials",
            "n_trials_phase",
            "of_total_trials_phase",
            "trials_per_capita",
            "n_trials_expected",
            "factor_deviation_n_trials_from_expected",
            "n_sites",
            "n_sites_phase",
            "of_total_sites_phase",
            "sites_per_capita",
            "n_sites_expected",
            "factor_deviation_n_sites_from_expected",
        ]
    ]
    for grouper_column_name in grouper_column_names
}

In [160]:
for grouper_column_name in grouper_column_names:
    display(consolidated_counts_per_phase[grouper_column_name].head())

,phase,country_ISO,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,AUS,25364307.0,12.0,64,0.187500,4.731058e-07,0.211929,56.622810,17.0,277.0,0.061372,6.702332e-07,0.917254,18.533579
1,PHASE2,AUS,25364307.0,10.0,123,0.081301,3.942548e-07,0.407301,24.551896,40.0,3432.0,0.011655,1.577019e-06,11.364678,3.519677
2,PHASE3,AUS,25364307.0,43.0,183,0.234973,1.695296e-06,0.605984,70.959004,175.0,15036.0,0.011639,6.899459e-06,49.790006,3.514762
3,PHASE4,AUS,25364307.0,9.0,79,0.113924,3.548293e-07,0.261600,34.403732,48.0,2567.0,0.018699,1.892423e-06,8.500329,5.646840
4,PHASE1,BEL,11484055.0,1.0,64,0.015625,8.707726e-08,0.095954,10.421684,1.0,277.0,0.003610,8.707726e-08,0.415300,2.407898


,phase,country_continent,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,Asia,4.555914e+09,6.0,64,0.093750,1.316970e-09,38.066449,0.157619,9.0,277.0,0.032491,1.975454e-09,164.756348,0.054626
1,PHASE2,Asia,4.555914e+09,35.0,123,0.284553,7.682322e-09,73.158956,0.478410,197.0,3432.0,0.057401,4.324050e-08,2041.313313,0.096506
2,PHASE3,Asia,4.555914e+09,92.0,183,0.502732,2.019353e-08,108.846252,0.845229,1156.0,15036.0,0.076882,2.537361e-07,8943.236297,0.129260
3,PHASE4,Asia,4.555914e+09,11.0,79,0.139241,2.414444e-09,46.988273,0.234101,61.0,2567.0,0.023763,1.338919e-08,1526.821467,0.039952
4,PHASE1,Europe,7.454125e+08,27.0,64,0.421875,3.622156e-08,6.228213,4.335112,63.0,277.0,0.227437,8.451697e-08,26.956485,2.337100


,phase,subregion,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,Australia and New Zealand,3.028131e+07,12.0,64,0.187500,3.962841e-07,0.253012,47.428545,17.0,277.0,0.061372,5.614025e-07,1.095068,15.524145
1,PHASE2,Australia and New Zealand,3.028131e+07,10.0,123,0.081301,3.302367e-07,0.486258,20.565223,45.0,3432.0,0.013112,1.486065e-06,13.567779,3.316681
2,PHASE3,Australia and New Zealand,3.028131e+07,47.0,183,0.256831,1.552113e-06,0.723457,64.965875,201.0,15036.0,0.013368,6.637758e-06,59.442052,3.381445
3,PHASE4,Australia and New Zealand,3.028131e+07,9.0,79,0.113924,2.972131e-07,0.312312,28.817344,49.0,2567.0,0.019088,1.618160e-06,10.148161,4.828461
4,PHASE1,Eastern Asia,1.628149e+09,2.0,64,0.031250,1.228389e-09,13.603821,0.147018,2.0,277.0,0.007220,1.228389e-09,58.879037,0.033968


,phase,economy_level,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,1. Developed region: G7,769418203.0,44.0,64,0.687500,5.718607e-08,6.428791,6.844211,222.0,277.0,0.801444,2.885297e-07,27.824610,7.978549
1,PHASE2,1. Developed region: G7,769418203.0,101.0,123,0.821138,1.312680e-07,12.355332,8.174608,1749.0,3432.0,0.509615,2.273146e-06,344.743902,5.073331
2,PHASE3,1. Developed region: G7,769418203.0,171.0,183,0.934426,2.222458e-07,18.382323,9.302415,6937.0,15036.0,0.461359,9.015903e-06,1510.364017,4.592933
3,PHASE4,1. Developed region: G7,769418203.0,60.0,79,0.759494,7.798100e-08,7.935539,7.560924,1902.0,2567.0,0.740943,2.471998e-06,257.854777,7.376245
4,PHASE1,2. Developed region: nonG7,352472908.0,21.0,64,0.328125,5.957905e-08,2.945049,7.130611,48.0,277.0,0.173285,1.361807e-07,12.746542,3.765727


,phase,consolidated_economy_level,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,1. Developed region,1.121891e+09,59.0,64,0.921875,5.258977e-08,9.373840,6.294112,270.0,277.0,0.974729,2.406651e-07,40.571151,6.654975
1,PHASE2,1. Developed region,1.121891e+09,118.0,123,0.959350,1.051795e-07,18.015349,6.549970,2863.0,3432.0,0.834207,2.551941e-06,502.672172,5.695561
2,PHASE3,1. Developed region,1.121891e+09,176.0,183,0.961749,1.568780e-07,26.803324,6.566350,11718.0,15036.0,0.779330,1.044486e-05,2202.266542,5.320882
3,PHASE4,1. Developed region,1.121891e+09,72.0,79,0.911392,6.417735e-08,11.570834,6.222542,2418.0,2567.0,0.941956,2.155289e-06,375.978865,6.431213
4,PHASE1,2. Emerging region,4.817026e+09,3.0,64,0.046875,6.227909e-10,40.248141,0.074538,3.0,277.0,0.010830,6.227909e-10,174.198986,0.017222


,phase,income_group,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,1. High income: OECD,1.074971e+09,59.0,64,0.921875,5.488522e-08,8.981801,6.568838,270.0,277.0,0.974729,2.511697e-07,38.874358,6.945452
1,PHASE2,1. High income: OECD,1.074971e+09,118.0,123,0.959350,1.097704e-07,17.261899,6.835864,2739.0,3432.0,0.798077,2.547977e-06,481.649084,5.686713
2,PHASE3,1. High income: OECD,1.074971e+09,176.0,183,0.961749,1.637254e-07,25.682338,6.852959,10896.0,15036.0,0.724661,1.013609e-05,2110.161896,5.163585
3,PHASE4,1. High income: OECD,1.074971e+09,71.0,79,0.898734,6.604832e-08,11.086911,6.403948,2400.0,2567.0,0.934944,2.232619e-06,360.254428,6.661958
4,PHASE1,3. Upper middle income,2.643713e+09,4.0,64,0.062500,1.513023e-09,22.089260,0.181083,4.0,277.0,0.014440,1.513023e-09,95.605077,0.041839


,phase,consolidated_income_group,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,1. High income,1.172607e+09,59.0,64,0.921875,5.031525e-08,9.797589,6.021890,270.0,277.0,0.974729,2.302562e-07,42.405190,6.367145
1,PHASE2,1. High income,1.172607e+09,118.0,123,0.959350,1.006305e-07,18.829741,6.266682,2752.0,3432.0,0.801865,2.346908e-06,525.395706,5.237957
2,PHASE3,1. High income,1.172607e+09,176.0,183,0.961749,1.500929e-07,28.014981,6.282353,11067.0,15036.0,0.736034,9.437947e-06,2301.821046,4.807932
3,PHASE4,1. High income,1.172607e+09,72.0,79,0.911392,6.140166e-08,12.093899,5.953415,2418.0,2567.0,0.941956,2.062072e-06,392.975168,6.153061
4,PHASE1,2. Upper middle income,2.643713e+09,4.0,64,0.062500,1.513023e-09,22.089260,0.181083,4.0,277.0,0.014440,1.513023e-09,95.605077,0.041839


,phase,hdi_category,population_estimate,n_trials,n_trials_phase,of_total_trials_phase,trials_per_capita,n_trials_expected,factor_deviation_n_trials_from_expected,n_sites,n_sites_phase,of_total_sites_phase,sites_per_capita,n_sites_expected,factor_deviation_n_sites_from_expected
0,PHASE1,1. Very high human development,1.640870e+09,60.0,64,0.937500,3.656597e-08,13.710112,4.376332,271.0,277.0,0.978339,1.651563e-07,59.339078,4.566974
1,PHASE2,1. Very high human development,1.640870e+09,121.0,123,0.983740,7.374137e-08,26.349121,4.592184,3110.0,3432.0,0.906177,1.895336e-06,735.204748,4.230114
2,PHASE3,1. Very high human development,1.640870e+09,180.0,183,0.983607,1.096979e-07,39.202351,4.591561,13065.0,15036.0,0.868915,7.962240e-06,3221.019401,4.056169
3,PHASE4,1. Very high human development,1.640870e+09,76.0,79,0.962025,4.631689e-08,16.923419,4.490818,2518.0,2567.0,0.980912,1.534552e-06,549.904017,4.578981
4,PHASE1,2. High human development,2.732477e+09,5.0,64,0.078125,1.829841e-09,22.830919,0.219001,6.0,277.0,0.021661,2.195810e-09,98.815071,0.060719


# Export data

In [ ]:
# Safety switch
polse

In [ ]:
with pd.ExcelWriter("data/results/trials_sites_counts.xlsx") as writer:
    # Base data
    trials_socioeconomic_data.to_excel(writer, sheet_name="Base_dataset", index=False)
    # Overall counts
    for grouper_column_name in grouper_column_names:
        consolidated_counts_overall[grouper_column_name].to_excel(
            writer, sheet_name="ovr_" + grouper_column_name, index=False
        )
    # Per phase counts
    for grouper_column_name in grouper_column_names:
        consolidated_counts_per_phase[grouper_column_name].to_excel(
            writer, sheet_name="phs_" + grouper_column_name, index=False
        )